In [ ]:
import sys
!{sys.executable} -m pip3 install pynput -i https://pypi.tuna.tsinghua.edu.cn/simple
from pynput import mouse

In [ ]:
import sys
from PyQt6.QtWidgets import (QApplication, QMessageBox, QWidget, QVBoxLayout, 
                            QLabel, QPushButton, QTextEdit, QCheckBox)
from PyQt6.QtCore import Qt, QTimer, QDateTime
from PyQt6.QtGui import QClipboard
from pynput import mouse

class GlobalMouseMonitor(QWidget):
    def __init__(self):
        super().__init__()
        self.init_ui()
        self.init_monitor()
        
    def init_ui(self):
        """初始化界面"""
        self.setWindowTitle("剪贴板监控")
        self.setWindowFlags(Qt.WindowType.WindowStaysOnTopHint)
        self.setMinimumSize(300, 400)
        
        # 创建布局
        layout = QVBoxLayout()
        
        # 状态显示
        self.status_label = QLabel("监控状态: 运行中")
        layout.addWidget(self.status_label)
        
        # 自动弹窗选项
        self.popup_checkbox = QCheckBox("复制时自动弹窗")
        self.popup_checkbox.setChecked(True)
        layout.addWidget(self.popup_checkbox)
        
        # 历史记录
        history_label = QLabel("复制历史:")
        layout.addWidget(history_label)
        
        self.history_text = QTextEdit()
        self.history_text.setReadOnly(True)
        layout.addWidget(self.history_text)
        
        # 清除按钮
        clear_button = QPushButton("清除历史")
        clear_button.clicked.connect(self.clear_history)
        layout.addWidget(clear_button)
        
        # 设置布局
        self.setLayout(layout)
        
    def init_monitor(self):
        """初始化监控器"""
        try:
            # 剪贴板相关
            self.clipboard = QApplication.clipboard()
            self.previous_clipboard_text = ""
            
            # 定时器
            self.clipboard_timer = QTimer()
            self.clipboard_timer.setInterval(200)  # 200ms
            self.clipboard_timer.timeout.connect(self.check_clipboard)
            self.clipboard_timer.start()
            
            # 鼠标监听器
            self.listener_thread = None
            self.setup_listener()
            
        except Exception as e:
            QMessageBox.critical(self, "错误", f"初始化失败: {str(e)}")
            self.close()
    
    def setup_listener(self):
        """设置鼠标监听器"""
        try:
            if not self.listener_thread:
                self.listener_thread = mouse.Listener(on_click=self.on_click)
                self.listener_thread.start()
                self.status_label.setText("监控状态: 运行中")
        except Exception as e:
            self.status_label.setText(f"监控状态: 错误 ({str(e)})")
    
    def on_click(self, x, y, button, pressed):
        """鼠标点击事件处理"""
        if not pressed and button == mouse.Button.left:
            QTimer.singleShot(500, self.check_clipboard_with_timeout)
    
    def check_clipboard_with_timeout(self):
        """检查剪贴板（带超时）"""
        try:
            current_text = self.clipboard.text()
            if current_text and current_text != self.previous_clipboard_text:
                self.previous_clipboard_text = current_text
                self.add_to_history(current_text)
                if self.popup_checkbox.isChecked():
                    self.show_popup(current_text)
        except Exception as e:
            self.status_label.setText(f"监控状态: 错误 ({str(e)})")
    
    def check_clipboard(self):
        """定时检查剪贴板 有新内容更新历史记录"""
        try:
            current_text = self.clipboard.text()
            if current_text and current_text != self.previous_clipboard_text:
                self.previous_clipboard_text = current_text
                self.add_to_history(current_text)
        except Exception as e:
            self.status_label.setText(f"监控状态: 错误 ({str(e)})")
    
    def add_to_history(self, text):
        """添加文本到历史记录"""
        try:
            current_time = QDateTime.currentDateTime().toString("yyyy-MM-dd hh:mm:ss")
            self.history_text.append(f"[{current_time}]\n{text}\n")
            
            # 自动滚动到底部
            self.history_text.verticalScrollBar().setValue(
                self.history_text.verticalScrollBar().maximum()
            )
        except Exception as e:
            self.status_label.setText(f"监控状态: 时间格式化错误 ({str(e)})")
    
    def show_popup(self, text):
        """显示弹窗"""
        QMessageBox.information(None, "新复制内容", f"复制的文字：\n{text}")
    
    def clear_history(self):
        """清除历史记录"""
        self.history_text.clear()
    
    def closeEvent(self, event):
        """关闭事件处理"""
        try:
            if self.listener_thread:
                self.listener_thread.stop()
            if self.clipboard_timer:
                self.clipboard_timer.stop()
            event.accept()
        except Exception as e:
            QMessageBox.warning(self, "警告", f"关闭时发生错误: {str(e)}")
            event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = GlobalMouseMonitor()
    window.show()
    sys.exit(app.exec())